### Поиск структуры в данных, неделя 1
#### Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.


Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

In [1]:
import pandas as pd
import numpy as np

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

In [2]:
df = pd.read_csv('checkins.csv')
print(df.shape)
df.head()

(1021967, 6)


C:\Users\admin\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,,,2012-04-21 17:42:58


In [3]:
# удаляем лишние пробелы в наименованиях столбцов
df.columns = df.columns.str.lstrip()
df.columns = df.columns.str.rstrip()
list(df)

['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at']

In [4]:
df.dtypes

id             object
user_id       float64
venue_id      float64
latitude       object
longitude      object
created_at     object
dtype: object

In [5]:
# меняем тип данных в столбцах 'latitude', 'longitude' - широта, долгота
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')
df.dropna(subset=['latitude', 'longitude'], inplace = True)
df.shape

(396634, 6)

In [6]:
# будем работать с первыми 100 тыс. строк
# для оптимизации качества кластеризации и времени, затраченного на ее построение, 
X = df[['latitude', 'longitude']][:100000]
X.shape

(100000, 2)

Необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Для решения воспользуемся алгоритмом MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [7]:
from sklearn.cluster import MeanShift

clustering = MeanShift(bandwidth=0.1).fit(X)

In [8]:
pred_X = pd.DataFrame(clustering.predict(X), columns=['cluster_label'])

In [10]:
# вычислим кол-во точек в каждом кластере
df_label = pred_X.groupby(['cluster_label']).size().reset_index()
df_label = df_label.rename(columns = {0:'count'})
df_label.head()

,cluster_label,count
0,0,12506
1,1,4692
2,2,3994
3,3,3363
4,4,3526


In [11]:
# объединим информацию о центре кластера и кол-ве точек в нем
cluster_centers = pd.DataFrame(clustering.cluster_centers_, columns=['latitude', 'longitude'])
df_centers = pd.concat([df_label, cluster_centers], axis = 1)
print(df_centers.shape)
df_centers.head()

(3230, 4)


,cluster_label,count,latitude,longitude
0,0,12506,40.717716,-73.991835
1,1,4692,33.449438,-112.002140
2,2,3994,33.446380,-111.901888
3,3,3363,41.878244,-87.629843
4,4,3526,37.688682,-122.409330


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [12]:
df_centers = df_centers[df_centers['count']>15]
df_centers = df_centers[['latitude', 'longitude']]
df_centers.shape

(593, 2)

In [33]:
df_centers.reset_index(drop = True, inplace = True)

In [13]:
# создаем df с координатами офисов компании
off_adr = [[33.751277, -118.188740], 
          [25.867736, -80.324116],
          [51.503016, -0.075479],
          [52.378894, 4.885084],
          [39.366487, 117.036146],
          [-33.868457, 151.205134]] 

off_adr = pd.DataFrame(off_adr, columns = ['latitude', 'longitude']) 
off_adr

,latitude,longitude
0,33.751277,-118.188740
1,25.867736,-80.324116
2,51.503016,-0.075479
3,52.378894,4.885084
4,39.366487,117.036146
5,-33.868457,151.205134


In [35]:
# рассчитываем расстояние от центра кластеров до каждого офиса компании
from scipy.spatial import distance

each_to_each_dist = []
for i in range(df_centers.shape[0]):
    for j in range(off_adr.shape[0]):
        each_to_each_dist.append([df_centers['latitude'][i], df_centers['longitude'][i], off_adr['latitude'][j], off_adr['longitude'][j], distance.euclidean(df_centers[i:i+1], off_adr[j:j+1])])


In [37]:
# определяем координаты для размещения 20 баннеров
fin = pd.DataFrame(each_to_each_dist, columns = ['latitude', 'longitude', 'off_latitude', 'off_longitude', 'distance'])
fin.sort_values(by='distance', ascending = True).head(20)

,latitude,longitude,off_latitude,off_longitude,distance
2465,-33.860630,151.204776,-33.868457,151.205134,0.007835
2235,52.372964,4.892317,52.378894,4.885084,0.009353
2413,25.845672,-80.318891,25.867736,-80.324116,0.022674
350,51.502991,-0.125537,51.503016,-0.075479,0.050058
306,33.809878,-118.148924,33.751277,-118.188740,0.070848
175,25.785812,-80.217938,25.867736,-80.324116,0.134109
997,25.705350,-80.283429,25.867736,-80.324116,0.167406
553,26.010098,-80.199991,25.867736,-80.324116,0.188876
522,33.888325,-118.048928,33.751277,-118.188740,0.195779
252,33.872986,-118.362091,33.751277,-118.188740,0.211811
